In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# <p style="background-color:#00008B;font-family:Comic Sans MS;font-size:100%;color:#f2ba49;text-align:center;border-radius:20px 20px;"><b>Import Libraries</b></p>

In [ ]:
import os
import re
import nltk
import string
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn import tree
from sklearn import metrics
from sklearn import ensemble
from sklearn import naive_bayes
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import feature_extraction

# <p style="background-color:#00008B;font-family:Comic Sans MS;font-size:100%;color:#f2ba49;text-align:center;border-radius:20px 20px;"><b>Data</b></p>

In [ ]:
file_path = "../input/language-detection/Language Detection.csv"
text_file = pd.read_csv(file_path, names = ['content', 'class'], skiprows=[0])

In [ ]:
text_file

In [ ]:
set(text_file['class'].to_numpy())

### LowerCase and Delete the Punctuations

In [ ]:
def cleaner(x):
  x = re.sub(r'[\([{})\]!@#$,"%^*?:;~`]', ' ', x)
  x = re.sub(r"\\|[0-9]|/|-|_|'|\.", '', x)
  x = re.sub('\s+', ' ', x)  
  x = x.lower()
  return x

text_file['content_1'] = text_file['content'].map(cleaner)

### Get X and Y

In [ ]:
X = text_file['content_1']
le = preprocessing.LabelEncoder()
Y = le.fit_transform(text_file['class'].to_numpy())

In [ ]:
le.classes_

In [ ]:
rand_data = [(i, j) for i, j in zip(np.array(X), Y)]
np.random.shuffle(rand_data)
rand_df = pd.DataFrame(rand_data, columns=['x', 'y'])

### Train/Test Split

with the distribution of each the class

In [ ]:
rand_df

In [ ]:
test_arr = []
train_arr = []

for i in range(len(le.classes_)):
  temp_df = rand_df[rand_df['y'] == i]
  df_size = len(temp_df)
  train_size =  df_size * 80 // 100
  train_arr.extend(temp_df[0: train_size].to_numpy())
  test_arr.extend(temp_df[train_size:].to_numpy())

In [ ]:
train_x, train_y = np.array([i[0] for i in train_arr]), np.array([i[1] for i in train_arr])
test_x, test_y = np.array([i[0] for i in test_arr]), np.array([i[1] for i in test_arr])

In [ ]:
train_x.shape, train_y.shape

In [ ]:
test_x.shape, test_y.shape

### Text Vectorization

In [ ]:
tf_idf_vectorizer = feature_extraction.text.TfidfVectorizer()

train_x = tf_idf_vectorizer.fit_transform(train_x)
test_x = tf_idf_vectorizer.transform(test_x)

# <p style="background-color:#00008B;font-family:Comic Sans MS;font-size:100%;color:#f2ba49;text-align:center;border-radius:20px 20px;"><b>Model Creation</b></p>

In [ ]:
dc_clf = tree.DecisionTreeClassifier()
edt_clf = tree.ExtraTreeClassifier()
mnb_clf = naive_bayes.MultinomialNB()
gnb_clf = naive_bayes.GaussianNB()
lsvc_clf = svm.LinearSVC()
svm_clf = svm.SVC()
rf_clf = ensemble.RandomForestClassifier()
et_clf = ensemble.ExtraTreesClassifier()
log_clf = linear_model.LogisticRegressionCV()
sgd_clf = linear_model.SGDClassifier()
ridge_clf = linear_model.RidgeClassifierCV()

In [ ]:
clf = [dc_clf,edt_clf,mnb_clf,gnb_clf,lsvc_clf,svm_clf,rf_clf,et_clf,log_clf,sgd_clf,ridge_clf]

In [ ]:
def data_spliter(x, y):
  test_arr = []
  train_arr = []
  rand_data = [(i, j) for i, j in zip(np.array(x), y)]
  np.random.shuffle(rand_data)
  rand_df = pd.DataFrame(rand_data, columns=['x', 'y'])
  for i in range(4):
    temp_df = rand_df[rand_df['y'] == i]
    df_size = len(temp_df)
    train_size =  df_size * 80 // 100
    train_arr.extend(temp_df[0: train_size].to_numpy())
    test_arr.extend(temp_df[train_size:].to_numpy())

  train_x, train_y = np.array([i[0] for i in train_arr]), np.array([i[1] for i in train_arr])
  test_x, test_y = np.array([i[0] for i in test_arr]), np.array([i[1] for i in test_arr])
  return train_x, train_y, test_x, test_y

# <p style="background-color:#00008B;font-family:Comic Sans MS;font-size:100%;color:#f2ba49;text-align:center;border-radius:20px 20px;"><b>Model Evaluation</b></p>

In [ ]:
iterations=10
vectorizer = feature_extraction.text.TfidfVectorizer()
arr = np.zeros((iterations, len(clf)))

for j in range(iterations):

  train_x, train_y, test_x, test_y = data_spliter(X, Y)
  train_x = vectorizer.fit_transform(train_x)
  test_x = vectorizer.transform(test_x)

  for c, classifier in enumerate(clf):
    classifier.fit(train_x.toarray(), train_y)
    pred = classifier.predict(test_x.toarray())
    arr[j][c] += metrics.accuracy_score(test_y, pred)

In [ ]:
mean_df = pd.DataFrame(arr.mean(axis=0).reshape(1, -1), columns=['DecisionTree','ExtraTree','MultinomialNB','GaussianNB', 'LinearSVC', 'SVC','RandomForest','ExtraRanTrees','LogisticRegression', 'SGDClassifier', 'RidgeClassifier'])

std_df = pd.DataFrame(arr.std(axis=0).reshape(1, -1), columns=['DecisionTree','ExtraTree','MultinomialNB','GaussianNB', 'LinearSVC', 'SVC','RandomForest','ExtraRanTrees','LogisticRegression', 'SGDClassifier', 'RidgeClassifier'])

In [ ]:
mean_df

In [ ]:
std_df